<a href="https://colab.research.google.com/github/cralji/Deep_Krein_RF/blob/main/Krein_Cat_dogs_Keras_Tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preliminares

## Intalar Paquetes Necesarios

In [1]:
%%capture
!pip install --upgrade keras==2.15.0
!pip install -U git+https://github.com/UN-GCPDS/python-gcpds.image_segmentation.git >> /tmp/null

In [2]:
%%capture
!pip install keras-tuner

In [3]:
%%capture
!git clone https://github.com/cralji/Deep_Krein_RF.git

## Incorporando Repo al PATH

In [4]:
import os
import sys

sys.path.append('/content/Deep_Krein_RF')

os.chdir('/content/Deep_Krein_RF')

## Cambiando KErnel del proceso Gaussiado de la Optimización Bayesiana dek KerasTuner a un Matern$_{\nu=\frac{1}{2}}$

In [5]:
import keras_tuner
import os

# Get the directory where keras_tuner is installed
keras_tuner_dir = os.path.dirname(keras_tuner.__file__)

# Construct the path to bayesian.py
bayesian_path = os.path.join(keras_tuner_dir, 'src', 'tuners', 'bayesian.py')

with open("cambio_matern_12.txt", "r") as f:
  new_content = f.read()
  print(new_content)

with open(bayesian_path, 'w') as file:
  file.write(new_content)

print(new_content)

# Copyright 2019 The KerasTuner Authors
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import numpy as np

try:
    import sklearn
    import sklearn.exceptions
    import sklearn.gaussian_process
except ImportError:  # pragma: no cover
    sklearn = None  # pragma: no cover

try:
    import scipy
    import scipy.optimize
except ImportError:  # pragma: no cover
    scipy = None  # pragma: no cover

from keras_tuner.src.api_export import keras_tuner_export
from keras_tuner.src.engine import hyp

In [6]:
import keras_tuner
import os

# Get the directory where keras_tuner is installed
keras_tuner_dir = os.path.dirname(keras_tuner.__file__)

# Construct the path to bayesian.py
bayesian_path = os.path.join(keras_tuner_dir, 'src', 'tuners', 'bayesian.py')

print(bayesian_path)

/usr/local/lib/python3.10/dist-packages/keras_tuner/src/tuners/bayesian.py


#Proceso

## Cargue Librerias

In [11]:
from krein_functions import *
from models import krein_rff_unet
from metrics import DiceCoefficient
from losses import WeightedDiceLoss

# General Libraries
import time
import shutil
import random
import warnings
import pandas as pd
import seaborn as sns
from enum import auto, Enum
from functools import partial
from datetime import datetime

# Image Processing Libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.axes import Axes
from matplotlib.cm import coolwarm

# Deep Learning Libraries
import tensorflow as tf
from tensorflow import keras
from keras_tuner import Objective
from keras_tuner import HyperModel
import tensorflow.keras.backend as K
from keras.layers import Layer, Activation
from keras_tuner import BayesianOptimization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import get_custom_objects
from keras_tuner.engine.hyperparameters import HyperParameters
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Machine Learning Libraries - Sklearn
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, DotProduct, ExpSineSquared, Matern

# Deep Learning Libraries - GCPDS
from gcpds.image_segmentation.datasets.segmentation import OxfordIiitPet

# Deep Learning Libraries - TensorFlow specific
# from tensorflow.keras.losses import Loss
# from tensorflow.keras.metrics import Metric
from tensorflow.keras import Model, layers, regularizers

# Other Libraries
import gc
import json
import gdown
import itertools
from PIL import ImageFont
from dataclasses import dataclass
from matplotlib.style import available
from tensorflow.python.framework.ops import EagerTensor

warnings.filterwarnings("ignore") # Disable warnings

## Load Dataset

In [ ]:
dataset = OxfordIiitPet()
train_dataset, val_dataset, test_dataset = dataset()

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

In [ ]:
def preprocess(img,mask):
    img = tf.image.resize(img,(256,256))
    mask = tf.image.resize(mask,(256,256))#Ch 1: Seed, Ch 2: No germinate, Ch 3: germinate
    mask = tf.cast(mask>0, tf.float32)
    mask = mask[...,1:]
    back_ground = tf.reduce_sum(mask, axis =[-1], keepdims=True)
    back_ground = back_ground == 0
    back_ground = tf.cast(back_ground, tf.float32)
    mask = tf.concat([mask,back_ground], axis=-1) #Ch 1: No germinate, Ch 2: germinate, Ch 3: Background
    return img,mask

train = train_dataset.map(lambda x,y,label,id:preprocess(x,y))
train = train_dataset.batch(64)
train = train_dataset.cache()
val = val_dataset.map(lambda x,y,label,id:preprocess(x,y))
val = val_dataset.batch(64)
val = val_dataset.cache()

# Guardar en Repo GitHub

In [ ]:
# path_token = "/content/drive/MyDrive/____UTP/tokens/token_colab.py"
# # Lee el token de tu Google Drive
# with open(path_token, 'r') as token_file:
#     token = token_file.read().strip()

# # Configura el comando git para usar el token
# os.system("git config --global user.email 'craljimenez@utp.edu.co'")
# os.system("git config --global user.name 'Cristian Jimenez Colab'")
# os.system(f"git remote set-url origin https://{token}@github.com/cralji/Deep_Krein_RF.git")

In [ ]:
# # Configurar Git (si es la primera vez)
# !git config --global user.email "craljimenez@utp.edu.co"
# !git config --global user.name "Cristian Jimenez Colab"

# # Hacer commit y push de los cambios
# !git add .
# !git commit -m "Actualización desde Colab"
# !git push origin main

# # Hacer pull de los cambios remotos
# !git pull origin main